In [2]:
import json
import pandas as pd
import numpy as np
import networkx as nx
import jellyfish
import os
import shutil
import subprocess
from scipy.cluster.hierarchy import dendrogram, linkage
from matplotlib import pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
from sklearn import preprocessing
from sklearn.cluster import AgglomerativeClustering

In [3]:
def read_dataframe_from_file(filename, filesubstr):
    
    final_results = []
    initial_line = 2 - 1
    initial_line_result = 5 - 1
    
    num_class_f = open('Final_Results/' + filesubstr + '/num_class.txt')
    for line in num_class_f:
        num_class = line
        
    #print(num_class)

    f = open(filename, 'r')
    tmp_arr = []
    for i, line in enumerate(f):
        if i == initial_line:
            line = line.replace('MoJo', '')
            line = line.replace('.rsf', '')
            line = line.replace('(', '')
            line = line.replace(')', '')
            line = line.split('_')
            name = line[0].split('/')[0]
            version = line[0].split('-')[-1]
            n_cluster = line[1]
            affinity = line[2]
            linkage = line[3]
            #print(line)
            #print('Version', version)
            initial_line += 5
            tmp_arr.append(name)
            tmp_arr.append(version)
            tmp_arr.append(n_cluster)
            tmp_arr.append(affinity)
            tmp_arr.append(linkage)
        elif i == initial_line_result:
            n_MoJo = line.split(' ')[-1].split('\n')[0]
            
            #print(line)
            initial_line_result += 5
            tmp_arr.append(n_MoJo)
            tmp_arr.append((1 - (float(n_MoJo) / float(num_class))) * 100)
            final_results.append(tmp_arr)
            tmp_arr = []
            
    
    
    f.close()
    return final_results

In [29]:
# Getting the smallest MoJo Value
rootdir = 'Final_Results/'
project_names = []

f = open('project_list.txt', 'r')
for line in f:
    #print(line)
    project_names.append(line.split('\n')[0])

print(project_names)
#project_names = ['apache-jmeter']
dir_arr = []

#Main clustering arr to be transformed to pandas dataframe later
cluster_main_arr = []

#Main CK arr to be transformed to pandas dataframe later
ck_sum_arr = []
ck_max_arr = []
ck_std_arr = []
ck_mean_arr = []

# Only considering the below ck_metrics
ck_metrics = ['cbo', 'wmc', 'dit', 'rfc', 'lcom',
       'totalMethods', 'staticMethods', 'publicMethods', 'privateMethods',
       'protectedMethods', 'defaultMethods', 'abstractMethods', 'finalMethods',
       'synchronizedMethods', 'totalFields', 'staticFields', 'publicFields',
       'privateFields', 'protectedFields', 'defaultFields', 'finalFields',
       'synchronizedFields', 'nosi', 'loc', 'returnQty', 'loopQty',
       'comparisonsQty', 'tryCatchQty', 'parenthesizedExpsQty',
       'stringLiteralsQty', 'numbersQty', 'assignmentsQty',
       'mathOperationsQty', 'variablesQty', 'maxNestedBlocks',
       'anonymousClassesQty', 'subClassesQty', 'lambdasQty', 'uniqueWordsQty',
       'modifiers']

depth = 3
for project_name in project_names:
    rootdir = 'Final_Results/'
    rootdir += project_name
    #print(rootdir)
    for root, dirs, files in os.walk(rootdir):
        for file in files:
            
            dir_arr.append(rootdir + '/' + str(file))
            
for element in dir_arr:
    #print(element)
    ver = str(element).replace('_class.csv', '')
    ver = ver.split('-')[-1]

    project_name_metrics = str(element).split('/')[1]

    if '_class.csv' in element:
        tmp_sum = []
        tmp_max = []
        tmp_std = []
        tmp_mean = []
        print(element)
        print(project_name_metrics)
        print(ver)
        try:
            df_sum = pd.read_csv(element)
            current_element_sum = df_sum.sum()

            df_max = pd.read_csv(element)
            current_element_max = df_max.max()

            df_std = pd.read_csv(element)
            current_element_std = df_std.std()

            df_mean = pd.read_csv(element)
            current_element_mean = df_mean.mean()



            tmp_sum.append(project_name_metrics)
            tmp_sum.append(ver)

            tmp_max.append(project_name_metrics)
            tmp_max.append(ver)

            tmp_std.append(project_name_metrics)
            tmp_std.append(ver)

            tmp_mean.append(project_name_metrics)
            tmp_mean.append(ver)

            for element in ck_metrics:
                tmp_sum.append(current_element_sum[element])
                tmp_max.append(current_element_max[element])
                tmp_std.append(current_element_std[element])
                tmp_mean.append(current_element_mean[element])

            ck_sum_arr.append(tmp_sum)
            ck_max_arr.append(tmp_max)
            ck_std_arr.append(tmp_std)
            ck_mean_arr.append(tmp_mean)
        except:
            print(element)

    elif '_results.txt' in element:
        #print(project_name, element)
        spark_df = pd.DataFrame(read_dataframe_from_file(element, project_name_metrics))
        spark_df.columns = ['name', 'version', 'n_clusters', 'affinity', 'linkage', 'n_MoJo', 'mojoFM']
        #spark_df['mojoFM'] = (1 - spark_df['n_MoJo'].astype(int)/spark_count)*100
        #print(spark_df[spark_df.n_MoJo == spark_df.n_MoJo.min()].iloc[0])
        #print(spark_df.loc[spark_df.groupby('version').mojoFM.idxmax()].reset_index(drop=True))
        
        for element in spark_df.iterrows():
            tmp_arr2 = []
            tmp_arr2.append(element[1]['name'])
            tmp_arr2.append(element[1]['version'])
            tmp_arr2.append(element[1]['n_clusters'])
            tmp_arr2.append(element[1]['affinity'])
            tmp_arr2.append(element[1]['linkage'])
            tmp_arr2.append(element[1]['n_MoJo'])
            tmp_arr2.append(element[1]['mojoFM'])

            cluster_main_arr.append(tmp_arr2)
#cluster_main_arr.append(spark_df.loc[spark_df.groupby('version').mojoFM.idxmax()].reset_index(drop=True))
        #cluster_main_arr.append(spark_df[spark_df.n_MoJo == spark_df.n_MoJo.min()].iloc[0])

rootdir = 'Final_Results/'
#dir_arr = []

            

['apache-storm', 'apache-cassandra', 'airbnb-lottie-android', 'apache-isis', 'apache-jmeter', 'apache-log4j', 'apache-maven', 'apache-spark', 'apache-tomcat', 'rzwitserloot-lombok', 'apache-tika', 'alibaba-fastjson', 'activiti-activiti', 'bumptech-glide', 'codecentric-spring-boot-admin', 'dropwizard-dropwizard', 'dropwizard-metrics', 'facebook-facebook-android-sdk', 'google-dagger', 'google-error-prone', 'grpc-grpc-java', 'java-native-accessjna', 'jenkinsci-jenkins', 'jhy-jsoup', 'mockito-mockito', 'mybatis-mybatis-3', 'naver-pinpoint', 'pxb1988-dex2jar', 'ReactiveX-RxJava', 'redisson-redisson', 'swagger-api-swagger-core', 'apache-accumulo', 'apache-activemq', 'apache-archiva', 'apache-avro', 'apache-beam', 'apache-bookkeeper', 'apache-calcite', 'scribejava-scribejava', 'joelittlejohn-jsonschema2pojo', 'robolectric-robolectric', 'rest-assured-rest-assured', 'mrniko-netty-socketio', 'JodaOrg-joda-time', 'hibernate-hibernate-orm', 'springfox-springfox', 'spring-projects-spring-security',

Final_Results/apache-spark/apache-spark-spark-2.3.0_class.csv
apache-spark
2.3.0
Final_Results/apache-spark/apache-spark-spark-2.3.1_class.csv
apache-spark
2.3.1
Final_Results/apache-spark/apache-spark-spark-2.3.2_class.csv
apache-spark
2.3.2
Final_Results/apache-spark/apache-spark-spark-2.3.3_class.csv
apache-spark
2.3.3
Final_Results/apache-spark/apache-spark-spark-2.3.4_class.csv
apache-spark
2.3.4
Final_Results/apache-spark/apache-spark-spark-2.4.0_class.csv
apache-spark
2.4.0
Final_Results/apache-spark/apache-spark-spark-2.4.1_class.csv
apache-spark
2.4.1
Final_Results/apache-spark/apache-spark-spark-2.4.2_class.csv
apache-spark
2.4.2
Final_Results/apache-spark/apache-spark-spark-2.4.3_class.csv
apache-spark
2.4.3
Final_Results/apache-spark/apache-spark-spark-2.4.4_class.csv
apache-spark
2.4.4
Final_Results/apache-tomcat/apache-tomcat-apache-tomcat-9.0.21_class.csv
apache-tomcat
9.0.21
Final_Results/apache-tomcat/apache-tomcat-apache-tomcat-9.0.22_class.csv
apache-tomcat
9.0.22
Fi

Final_Results/dropwizard-dropwizard/dropwizard-dropwizard-dropwizard-dropwizard-1.3.14_class.csv
dropwizard-dropwizard
1.3.14
Final_Results/dropwizard-dropwizard/dropwizard-dropwizard-dropwizard-dropwizard-1.3.2_class.csv
dropwizard-dropwizard
1.3.2
Final_Results/dropwizard-dropwizard/dropwizard-dropwizard-dropwizard-dropwizard-1.3.3_class.csv
dropwizard-dropwizard
1.3.3
Final_Results/dropwizard-dropwizard/dropwizard-dropwizard-dropwizard-dropwizard-1.3.4_class.csv
dropwizard-dropwizard
1.3.4
Final_Results/dropwizard-dropwizard/dropwizard-dropwizard-dropwizard-dropwizard-1.3.5_class.csv
dropwizard-dropwizard
1.3.5
Final_Results/dropwizard-dropwizard/dropwizard-dropwizard-dropwizard-dropwizard-1.3.6_class.csv
dropwizard-dropwizard
1.3.6
Final_Results/dropwizard-dropwizard/dropwizard-dropwizard-dropwizard-dropwizard-1.3.7_class.csv
dropwizard-dropwizard
1.3.7
Final_Results/dropwizard-dropwizard/dropwizard-dropwizard-dropwizard-dropwizard-1.3.8_class.csv
dropwizard-dropwizard
1.3.8
Final_

Final_Results/jenkinsci-jenkins/jenkinsci-jenkins-jenkinsci-jenkins-2.187_class.csv
jenkinsci-jenkins
2.187
Final_Results/jenkinsci-jenkins/jenkinsci-jenkins-jenkinsci-jenkins-2.189_class.csv
jenkinsci-jenkins
2.189
Final_Results/jenkinsci-jenkins/jenkinsci-jenkins-jenkinsci-jenkins-2.190_class.csv
jenkinsci-jenkins
2.190
Final_Results/jenkinsci-jenkins/jenkinsci-jenkins-jenkinsci-jenkins-2.191_class.csv
jenkinsci-jenkins
2.191
Final_Results/jenkinsci-jenkins/jenkinsci-jenkins-jenkinsci-jenkins-2.192_class.csv
jenkinsci-jenkins
2.192
Final_Results/jenkinsci-jenkins/jenkinsci-jenkins-jenkinsci-jenkins-2.193_class.csv
jenkinsci-jenkins
2.193
Final_Results/jenkinsci-jenkins/jenkinsci-jenkins-jenkinsci-jenkins-2.194_class.csv
jenkinsci-jenkins
2.194
Final_Results/jenkinsci-jenkins/jenkinsci-jenkins-jenkinsci-jenkins-2.195_class.csv
jenkinsci-jenkins
2.195
Final_Results/jhy-jsoup/jhy-jsoup-jhy-jsoup-1.6.1_class.csv
jhy-jsoup
1.6.1
Final_Results/jhy-jsoup/jhy-jsoup-jhy-jsoup-1.6.2_class.csv


Final_Results/swagger-api-swagger-core/swagger-api-swagger-core-swagger-api-swagger-core-2.0.5_class.csv
swagger-api-swagger-core
2.0.5
Final_Results/swagger-api-swagger-core/swagger-api-swagger-core-swagger-api-swagger-core-2.0.6_class.csv
swagger-api-swagger-core
2.0.6
Final_Results/swagger-api-swagger-core/swagger-api-swagger-core-swagger-api-swagger-core-2.0.7_class.csv
swagger-api-swagger-core
2.0.7
Final_Results/swagger-api-swagger-core/swagger-api-swagger-core-swagger-api-swagger-core-2.0.8_class.csv
swagger-api-swagger-core
2.0.8
Final_Results/swagger-api-swagger-core/swagger-api-swagger-core-swagger-api-swagger-core-2.0.9_class.csv
swagger-api-swagger-core
2.0.9
Final_Results/apache-accumulo/apache-accumulo-apache-accumulo-1.7.2_class.csv
apache-accumulo
1.7.2
Final_Results/apache-accumulo/apache-accumulo-apache-accumulo-1.7.3_class.csv
apache-accumulo
1.7.3
Final_Results/apache-accumulo/apache-accumulo-apache-accumulo-1.7.4_class.csv
apache-accumulo
1.7.4
Final_Results/apache

Final_Results/joelittlejohn-jsonschema2pojo/joelittlejohn-jsonschema2pojo-joelittlejohn-jsonschema2pojo-0.4.32_class.csv
joelittlejohn-jsonschema2pojo
0.4.32
Final_Results/joelittlejohn-jsonschema2pojo/joelittlejohn-jsonschema2pojo-joelittlejohn-jsonschema2pojo-0.4.33_class.csv
joelittlejohn-jsonschema2pojo
0.4.33
Final_Results/joelittlejohn-jsonschema2pojo/joelittlejohn-jsonschema2pojo-joelittlejohn-jsonschema2pojo-0.4.34_class.csv
joelittlejohn-jsonschema2pojo
0.4.34
Final_Results/joelittlejohn-jsonschema2pojo/joelittlejohn-jsonschema2pojo-joelittlejohn-jsonschema2pojo-0.4.35_class.csv
joelittlejohn-jsonschema2pojo
0.4.35
Final_Results/joelittlejohn-jsonschema2pojo/joelittlejohn-jsonschema2pojo-joelittlejohn-jsonschema2pojo-0.4.36_class.csv
joelittlejohn-jsonschema2pojo
0.4.36
Final_Results/joelittlejohn-jsonschema2pojo/joelittlejohn-jsonschema2pojo-joelittlejohn-jsonschema2pojo-0.4.37_class.csv
joelittlejohn-jsonschema2pojo
0.4.37
Final_Results/joelittlejohn-jsonschema2pojo/joelittl

Final_Results/springfox-springfox/springfox-springfox-springfox-springfox-2.3.1_class.csv
springfox-springfox
2.3.1
Final_Results/springfox-springfox/springfox-springfox-springfox-springfox-2.4.0_class.csv
springfox-springfox
2.4.0
Final_Results/springfox-springfox/springfox-springfox-springfox-springfox-2.5.0_class.csv
springfox-springfox
2.5.0
Final_Results/springfox-springfox/springfox-springfox-springfox-springfox-2.6.0_class.csv
springfox-springfox
2.6.0
Final_Results/springfox-springfox/springfox-springfox-springfox-springfox-2.6.1_class.csv
springfox-springfox
2.6.1
Final_Results/springfox-springfox/springfox-springfox-springfox-springfox-2.7.0_class.csv
springfox-springfox
2.7.0
Final_Results/springfox-springfox/springfox-springfox-springfox-springfox-2.8.0_class.csv
springfox-springfox
2.8.0
Final_Results/springfox-springfox/springfox-springfox-springfox-springfox-2.9.0_class.csv
springfox-springfox
2.9.0
Final_Results/springfox-springfox/springfox-springfox-springfox-springfo

Final_Results/knowm-XChange/knowm-XChange-knowm-XChange-4.3.7_class.csv
knowm-XChange
4.3.7
Final_Results/knowm-XChange/knowm-XChange-knowm-XChange-4.3.8_class.csv
knowm-XChange
4.3.8
Final_Results/knowm-XChange/knowm-XChange-knowm-XChange-4.3.9_class.csv
knowm-XChange
4.3.9
Final_Results/knowm-XChange/knowm-XChange-knowm-XChange-4.4.0_class.csv
knowm-XChange
4.4.0
Final_Results/hazelcast-hazelcast/hazelcast-hazelcast-hazelcast-hazelcast-3.11.4_class.csv
hazelcast-hazelcast
3.11.4
Final_Results/hazelcast-hazelcast/hazelcast-hazelcast-hazelcast-hazelcast-3.12.1_class.csv
hazelcast-hazelcast
3.12.1
Final_Results/hazelcast-hazelcast/hazelcast-hazelcast-hazelcast-hazelcast-3.12.2_class.csv
hazelcast-hazelcast
3.12.2
Final_Results/hazelcast-hazelcast/hazelcast-hazelcast-hazelcast-hazelcast-3.12_class.csv
hazelcast-hazelcast
3.12
Final_Results/hazelcast-hazelcast/hazelcast-hazelcast-hazelcast-hazelcast-3.8.9_class.csv
hazelcast-hazelcast
3.8.9
Final_Results/hazelcast-hazelcast/hazelcast-haze

Final_Results/oracle-opengrok/oracle-opengrok-oracle-opengrok-1.2.9_class.csv
oracle-opengrok
1.2.9
Final_Results/oracle-opengrok/oracle-opengrok-oracle-opengrok-1.3.0_class.csv
oracle-opengrok
1.3.0
Final_Results/oracle-opengrok/oracle-opengrok-oracle-opengrok-1.3.1_class.csv
oracle-opengrok
1.3.1
Final_Results/oracle-opengrok/oracle-opengrok-oracle-opengrok-1.3.2_class.csv
oracle-opengrok
1.3.2
Final_Results/lettuce-io-lettuce-core/lettuce-io-lettuce-core-lettuce-io-lettuce-core-5.1.0_class.csv
lettuce-io-lettuce-core
5.1.0
Final_Results/lettuce-io-lettuce-core/lettuce-io-lettuce-core-lettuce-io-lettuce-core-5.1.1_class.csv
lettuce-io-lettuce-core
5.1.1
Final_Results/lettuce-io-lettuce-core/lettuce-io-lettuce-core-lettuce-io-lettuce-core-5.1.2_class.csv
lettuce-io-lettuce-core
5.1.2
Final_Results/lettuce-io-lettuce-core/lettuce-io-lettuce-core-lettuce-io-lettuce-core-5.1.3_class.csv
lettuce-io-lettuce-core
5.1.3
Final_Results/lettuce-io-lettuce-core/lettuce-io-lettuce-core-lettuce-io

Final_Results/ionic-team-capacitor/ionic-team-capacitor-ionic-team-capacitor-1.1.1_class.csv
ionic-team-capacitor
1.1.1
Final_Results/ionic-team-capacitor/ionic-team-capacitor-ionic-team-capacitor-1.2.0_class.csv
ionic-team-capacitor
1.2.0
Final_Results/ionic-team-capacitor/ionic-team-capacitor-ionic-team-capacitor-1.2.1_class.csv
ionic-team-capacitor
1.2.1
Final_Results/cglib-cglib/cglib-cglib-cglib-cglib-3.2.12_class.csv
cglib-cglib
3.2.12
Final_Results/cglib-cglib/cglib-cglib-cglib-cglib-3.2.2_class.csv
cglib-cglib
3.2.2
Final_Results/cglib-cglib/cglib-cglib-cglib-cglib-3.2.3_class.csv
cglib-cglib
3.2.3
Final_Results/cglib-cglib/cglib-cglib-cglib-cglib-3.2.4_class.csv
cglib-cglib
3.2.4
Final_Results/cglib-cglib/cglib-cglib-cglib-cglib-3.2.5_class.csv
cglib-cglib
3.2.5
Final_Results/cglib-cglib/cglib-cglib-cglib-cglib-3.2.6_class.csv
cglib-cglib
3.2.6
Final_Results/cglib-cglib/cglib-cglib-cglib-cglib-3.2.7_class.csv
cglib-cglib
3.2.7
Final_Results/cglib-cglib/cglib-cglib-cglib-cglib-

### For clustering

In [30]:
final_clustering_result = pd.DataFrame(cluster_main_arr)
final_clustering_result.columns = ['name', 'version', 'n_clusters', 'affinity', 'linkage', 'n_MoJo', 'mojoFM']

In [34]:
final_clustering_result.sort_values('mojoFM', ascending=False).head(30)

,name,version,n_clusters,affinity,linkage,n_MoJo,mojoFM
6005,apache-spark,2.4.3,50,l1,single,107,99.228217
6011,apache-spark,2.4.3,50,manhattan,single,112,99.192152
5915,apache-spark,2.4.2,50,l1,single,112,99.192152
5921,apache-spark,2.4.2,50,manhattan,single,113,99.184939
6101,apache-spark,2.4.4,50,manhattan,single,115,99.170514
6095,apache-spark,2.4.4,50,l1,single,115,99.170514
6094,apache-spark,2.4.4,50,l1,average,123,99.112810
6100,apache-spark,2.4.4,50,manhattan,average,127,99.083958
6010,apache-spark,2.4.3,50,manhattan,average,128,99.076746
6004,apache-spark,2.4.3,50,l1,average,128,99.076746


In [35]:
print(len(final_clustering_result['name'].unique()))
final_clustering_result['name'].unique()


75


array(['apache-storm', 'apache-cassandra', 'airbnb-lottie-android',
       'apache-isis', 'apache-jmeter', 'apache-log4j', 'apache-maven',
       'apache-spark', 'apache-tomcat', 'rzwitserloot-lombok',
       'apache-tika', 'alibaba-fastjson', 'activiti-activiti',
       'bumptech-glide', 'codecentric-spring-boot-admin',
       'dropwizard-dropwizard', 'dropwizard-metrics',
       'facebook-facebook-android-sdk', 'google-dagger',
       'google-error-prone', 'grpc-grpc-java', 'java-native-accessjna',
       'jenkinsci-jenkins', 'jhy-jsoup', 'mockito-mockito',
       'mybatis-mybatis-3', 'naver-pinpoint', 'pxb1988-dex2jar',
       'ReactiveX-RxJava', 'redisson-redisson',
       'swagger-api-swagger-core', 'apache-accumulo', 'apache-activemq',
       'apache-archiva', 'apache-avro', 'apache-beam',
       'apache-bookkeeper', 'apache-calcite', 'scribejava-scribejava',
       'joelittlejohn-jsonschema2pojo', 'robolectric-robolectric',
       'rest-assured-rest-assured', 'mrniko-netty-socke

### For producing temp output files

Temp Output files
1. clustering_result_10_ver
2. ck_sum
3. ck_max
4. ck_std
5. ck_mean

In [36]:
final_clustering_result.to_csv('full_clustering_result_75_ver.csv', index=False)

### For CK Metrics

In [37]:
# For SUM CK Metric

ck_sum_df = pd.DataFrame(ck_sum_arr)
final_ck_columns = ['project_name', 'version']
for element in ck_metrics:
    final_ck_columns.append(element)
ck_sum_df.columns = final_ck_columns
ck_sum_df.head()

ck_sum_df.to_csv('ck_sum_75_ver.csv', index=False)

In [38]:
# For Max CK Metric

ck_max_df = pd.DataFrame(ck_max_arr)
final_ck_columns = ['project_name', 'version']
for element in ck_metrics:
    final_ck_columns.append(element)
ck_max_df.columns = final_ck_columns
ck_max_df.head()

ck_max_df.to_csv('ck_max_75_ver.csv', index=False)

In [39]:
# For SUM CK Metric

ck_std_df = pd.DataFrame(ck_std_arr)
final_ck_columns = ['project_name', 'version']
for element in ck_metrics:
    final_ck_columns.append(element)
ck_std_df.columns = final_ck_columns
ck_std_df.head()

ck_std_df.to_csv('ck_std_75_ver.csv', index=False)

In [40]:
# For SUM CK Metric

ck_mean_df = pd.DataFrame(ck_mean_arr)
final_ck_columns = ['project_name', 'version']
for element in ck_metrics:
    final_ck_columns.append(element)
ck_mean_df.columns = final_ck_columns
ck_mean_df.head()

ck_mean_df.to_csv('ck_mean_75_ver.csv', index=False)

## Start here if just looking for pre processing

In [6]:
import pandas as pd
final_clustering_result = pd.read_csv('full_clustering_result_30_ver.csv')
ck_sum_df = pd.read_csv('ck_sum_30_ver.csv')
ck_max_df = pd.read_csv('ck_max_30_ver.csv')
ck_std_df = pd.read_csv('ck_std_30_ver.csv')
ck_mean_df = pd.read_csv('ck_mean_30_ver.csv')


### To convert to Dr Aldeida's format

In [41]:
final_clustering_result['Instance'] = final_clustering_result['name'] + '_' + final_clustering_result['version']
final_clustering_result['n_clusters_str'] = final_clustering_result['n_clusters'].astype(str)
final_clustering_result = final_clustering_result.drop(columns=['n_clusters'])
final_clustering_result['Label'] = final_clustering_result['affinity']  

In [42]:
final_clustering_result = pd.pivot_table(final_clustering_result, values='mojoFM', index=['Instance'], columns=['Label'], aggfunc=np.mean).reset_index()

In [43]:
final_clustering_result.head(100)

Label,Instance,cosine,euclidean,l1,l2,manhattan
0,JodaOrg-joda-time_2.8.2,67.407407,68.379630,71.620370,68.888889,71.539352
1,JodaOrg-joda-time_2.9,67.048611,68.935185,71.840278,69.074074,72.152778
2,JodaOrg-joda-time_2.9.1,67.986111,68.356481,71.111111,68.310185,71.226852
3,JodaOrg-joda-time_2.9.2,68.020833,68.854167,70.810185,69.131944,70.636574
4,JodaOrg-joda-time_2.9.3,67.557870,68.530093,72.361111,68.738426,72.118056
5,JodaOrg-joda-time_2.9.4,67.037037,68.541667,72.581019,68.171296,71.747685
6,JodaOrg-joda-time_2.9.5,67.453704,68.900463,72.222222,68.877315,72.384259
7,JodaOrg-joda-time_2.9.6,68.298611,68.761574,70.682870,69.074074,70.613426
8,JodaOrg-joda-time_2.9.7,68.587963,69.363426,70.925926,69.120370,70.787037
9,JodaOrg-joda-time_2.9.8,67.210648,68.900463,71.875000,68.819444,71.921296


In [44]:
ck_sum_df['Instance'] = ck_sum_df['project_name'] + '_' + ck_sum_df['version']
ck_max_df['Instance'] = ck_max_df['project_name'] + '_' + ck_max_df['version']

#ck_sum_df.head()

temp1 = ck_sum_df.merge(ck_max_df, on='Instance', suffixes=('_sum', '_max') )

ck_std_df['Instance'] = ck_std_df['project_name'] + '_' + ck_std_df['version']
ck_mean_df['Instance'] = ck_mean_df['project_name'] + '_' + ck_mean_df['version']

#ck_std_df.head()

temp2 = ck_std_df.merge(ck_mean_df, on='Instance', suffixes=('_std', '_mean') )

temp1 = temp1.merge(temp2, on='Instance')
final_clustering_result = final_clustering_result.merge(temp1, on='Instance')

### Change below for visualization

1. Label = n_clusters + affinity + linkage
2. Label = affinity
3. Label = linkage

In [45]:
#final_clustering_result['Label'] = final_clustering_result['n_clusters_str'] + '_' + final_clustering_result['affinity'] + '_' +final_clustering_result['linkage'] 
#final_clustering_result['Label'] =  final_clustering_result['linkage'] 

In [46]:
columns_to_avoid = ['name', 'version', 'n_clusters', 'affinity', 'linkage', 'n_MoJo']
total_cancer = []
for element in final_clustering_result.columns:
    #print(element)
    for cancer in columns_to_avoid:
        print(cancer)
        if cancer in element:
            total_cancer.append(element)
            
total_cancer

name
version
n_clusters
affinity
linkage
n_MoJo
name
version
n_clusters
affinity
linkage
n_MoJo
name
version
n_clusters
affinity
linkage
n_MoJo
name
version
n_clusters
affinity
linkage
n_MoJo
name
version
n_clusters
affinity
linkage
n_MoJo
name
version
n_clusters
affinity
linkage
n_MoJo
name
version
n_clusters
affinity
linkage
n_MoJo
name
version
n_clusters
affinity
linkage
n_MoJo
name
version
n_clusters
affinity
linkage
n_MoJo
name
version
n_clusters
affinity
linkage
n_MoJo
name
version
n_clusters
affinity
linkage
n_MoJo
name
version
n_clusters
affinity
linkage
n_MoJo
name
version
n_clusters
affinity
linkage
n_MoJo
name
version
n_clusters
affinity
linkage
n_MoJo
name
version
n_clusters
affinity
linkage
n_MoJo
name
version
n_clusters
affinity
linkage
n_MoJo
name
version
n_clusters
affinity
linkage
n_MoJo
name
version
n_clusters
affinity
linkage
n_MoJo
name
version
n_clusters
affinity
linkage
n_MoJo
name
version
n_clusters
affinity
linkage
n_MoJo
name
version
n_clusters
affinity
linkage

['project_name_sum',
 'version_sum',
 'project_name_max',
 'version_max',
 'project_name_std',
 'version_std',
 'project_name_mean',
 'version_mean']

In [47]:
final_clustering_result = final_clustering_result.drop(columns=total_cancer)

In [48]:
final_clustering_result.head(100)

,Instance,cosine,euclidean,l1,l2,manhattan,cbo_sum,wmc_sum,dit_sum,rfc_sum,...,numbersQty_mean,assignmentsQty_mean,mathOperationsQty_mean,variablesQty_mean,maxNestedBlocks_mean,anonymousClassesQty_mean,subClassesQty_mean,lambdasQty_mean,uniqueWordsQty_mean,modifiers_mean
0,JodaOrg-joda-time_2.9,67.048611,68.935185,71.840278,69.074074,72.152778,4154,12881,896,10798,...,143.708207,46.668693,13.872340,34.100304,1.100304,0.237082,0.440729,0.000000,128.674772,109.389058
1,JodaOrg-joda-time_2.9.1,67.986111,68.356481,71.111111,68.310185,71.226852,4154,12883,896,10799,...,143.762918,46.680851,13.884498,34.112462,1.100304,0.237082,0.440729,0.000000,128.720365,109.389058
2,JodaOrg-joda-time_2.9.2,68.020833,68.854167,70.810185,69.131944,70.636574,4154,12892,896,10801,...,144.182371,46.723404,13.890578,34.155015,1.094225,0.237082,0.440729,0.000000,128.753799,109.389058
3,JodaOrg-joda-time_2.9.3,67.557870,68.530093,72.361111,68.738426,72.118056,4154,12895,896,10801,...,144.282675,46.735562,13.902736,34.158055,1.094225,0.237082,0.440729,0.000000,128.756839,109.389058
4,JodaOrg-joda-time_2.9.4,67.037037,68.541667,72.581019,68.171296,71.747685,4154,12895,896,10803,...,144.282675,46.741641,13.902736,34.167173,1.094225,0.243161,0.440729,0.000000,128.778116,109.389058
5,JodaOrg-joda-time_2.9.5,67.453704,68.900463,72.222222,68.877315,72.384259,4153,12929,896,10824,...,144.592705,46.917933,13.902736,34.325228,1.094225,0.243161,0.440729,0.000000,128.902736,109.389058
6,JodaOrg-joda-time_2.9.6,68.298611,68.761574,70.682870,69.074074,70.613426,4153,12935,896,10832,...,144.607903,46.942249,13.896657,34.343465,1.094225,0.243161,0.440729,0.000000,128.920973,109.389058
7,JodaOrg-joda-time_2.9.7,68.587963,69.363426,70.925926,69.120370,70.787037,4153,12935,896,10832,...,144.607903,46.942249,13.896657,34.343465,1.094225,0.243161,0.440729,0.000000,128.936170,109.389058
8,JodaOrg-joda-time_2.9.8,67.210648,68.900463,71.875000,68.819444,71.921296,4155,12940,896,10839,...,145.048632,46.963526,13.930091,34.361702,1.094225,0.243161,0.443769,0.000000,128.951368,109.389058
9,ReactiveX-RxJava_2.2.11,60.094851,55.039360,51.290489,55.071622,51.509872,22033,21123,3916,30019,...,22.323547,14.796330,1.023853,14.258104,0.799388,5.209174,0.688073,0.000000,60.351682,37.881346


In [49]:
temp_results = final_clustering_result[['cosine', 'euclidean', 'l1', 'l2', 'manhattan']]
final_clustering_result = final_clustering_result.drop(columns=['cosine', 'euclidean', 'l1', 'l2', 'manhattan'])
final_clustering_result = final_clustering_result.join(temp_results)

In [50]:
final_clustering_result.head()

,Instance,cbo_sum,wmc_sum,dit_sum,rfc_sum,lcom_sum,totalMethods_sum,staticMethods_sum,publicMethods_sum,privateMethods_sum,...,anonymousClassesQty_mean,subClassesQty_mean,lambdasQty_mean,uniqueWordsQty_mean,modifiers_mean,cosine,euclidean,l1,l2,manhattan
0,JodaOrg-joda-time_2.9,4154,12881,896,10798,158923,8738,740,7600,325,...,0.237082,0.440729,0.0,128.674772,109.389058,67.048611,68.935185,71.840278,69.074074,72.152778
1,JodaOrg-joda-time_2.9.1,4154,12883,896,10799,158940,8739,740,7601,325,...,0.237082,0.440729,0.0,128.720365,109.389058,67.986111,68.356481,71.111111,68.310185,71.226852
2,JodaOrg-joda-time_2.9.2,4154,12892,896,10801,159164,8745,740,7607,325,...,0.237082,0.440729,0.0,128.753799,109.389058,68.020833,68.854167,70.810185,69.131944,70.636574
3,JodaOrg-joda-time_2.9.3,4154,12895,896,10801,159251,8746,740,7608,325,...,0.237082,0.440729,0.0,128.756839,109.389058,67.557870,68.530093,72.361111,68.738426,72.118056
4,JodaOrg-joda-time_2.9.4,4154,12895,896,10803,159239,8747,740,7609,325,...,0.243161,0.440729,0.0,128.778116,109.389058,67.037037,68.541667,72.581019,68.171296,71.747685


In [52]:
final_clustering_result.to_csv('full_75_linkage_results.csv', index=False)

In [84]:
for element in final_clustering_result.columns:
    print(element)

cbo_sum
wmc_sum
dit_sum
rfc_sum
lcom_sum
totalMethods_sum
staticMethods_sum
publicMethods_sum
privateMethods_sum
protectedMethods_sum
defaultMethods_sum
abstractMethods_sum
finalMethods_sum
synchronizedMethods_sum
totalFields_sum
staticFields_sum
publicFields_sum
privateFields_sum
protectedFields_sum
defaultFields_sum
finalFields_sum
synchronizedFields_sum
nosi_sum
loc_sum
returnQty_sum
loopQty_sum
comparisonsQty_sum
tryCatchQty_sum
parenthesizedExpsQty_sum
stringLiteralsQty_sum
numbersQty_sum
assignmentsQty_sum
mathOperationsQty_sum
variablesQty_sum
maxNestedBlocks_sum
anonymousClassesQty_sum
subClassesQty_sum
lambdasQty_sum
uniqueWordsQty_sum
modifiers_sum
Instance
cbo_max
wmc_max
dit_max
rfc_max
lcom_max
totalMethods_max
staticMethods_max
publicMethods_max
privateMethods_max
protectedMethods_max
defaultMethods_max
abstractMethods_max
finalMethods_max
synchronizedMethods_max
totalFields_max
staticFields_max
publicFields_max
privateFields_max
protectedFields_max
defaultFields_max
fina

In [42]:
final_clustering_result.to_csv('remodularization_input_30_linkage.csv', index=False)

In [43]:
final_clustering_result.head()

,mojoFM,Instance,cbo_sum,wmc_sum,dit_sum,rfc_sum,lcom_sum,totalMethods_sum,staticMethods_sum,publicMethods_sum,...,assignmentsQty_mean,mathOperationsQty_mean,variablesQty_mean,maxNestedBlocks_mean,anonymousClassesQty_mean,subClassesQty_mean,lambdasQty_mean,uniqueWordsQty_mean,modifiers_mean,Label
0,21.849148,apache-storm_1.1.1,13675,27066,3097,23667,117111,13332,1528,10868,...,392,57,333,7,45,593,0,1178,1025,50_euclidean_complete
1,20.486618,apache-storm_1.1.1,13675,27066,3097,23667,117111,13332,1528,10868,...,392,57,333,7,45,593,0,1178,1025,50_euclidean_average
2,23.406326,apache-storm_1.1.1,13675,27066,3097,23667,117111,13332,1528,10868,...,392,57,333,7,45,593,0,1178,1025,50_euclidean_single
3,21.070560,apache-storm_1.1.1,13675,27066,3097,23667,117111,13332,1528,10868,...,392,57,333,7,45,593,0,1178,1025,50_l1_complete
4,21.265207,apache-storm_1.1.1,13675,27066,3097,23667,117111,13332,1528,10868,...,392,57,333,7,45,593,0,1178,1025,50_l1_average


In [48]:
test1 = test1.drop(columns=['Label'])

In [49]:
test1.head()

,Instance,cbo_sum,wmc_sum,dit_sum,rfc_sum,lcom_sum,totalMethods_sum,staticMethods_sum,publicMethods_sum,privateMethods_sum,...,numbersQty_mean,assignmentsQty_mean,mathOperationsQty_mean,variablesQty_mean,maxNestedBlocks_mean,anonymousClassesQty_mean,subClassesQty_mean,lambdasQty_mean,uniqueWordsQty_mean,modifiers_mean
0,apache-storm_1.1.1,13675,27066,3097,23667,117111,13332,1528,10868,1235,...,613,392,57,333,7,45,593,0,1178,1025
1,apache-storm_1.1.2,13930,27294,3120,24140,117459,13488,1530,10990,1248,...,613,392,57,333,7,45,593,0,1187,1025
2,apache-storm_1.1.3,13974,27385,3128,24221,118054,13522,1549,11006,1263,...,613,392,53,333,7,45,593,0,1191,1025
3,apache-storm_1.1.4,13975,27391,3128,24209,118196,13524,1551,11010,1262,...,613,392,53,333,7,45,593,0,1191,1025
4,apache-storm_1.2.0,14345,28032,3194,24958,119489,13964,1589,11303,1340,...,613,392,57,333,7,45,593,0,1209,1025


In [23]:
agglo = pd.read_csv('clustering_result_30_ver.csv')
bunch = pd.read_csv('bunch_clustering_result_30_ver.csv')

ck_sum_df = pd.read_csv('ck_sum_30_ver.csv')
ck_max_df = pd.read_csv('ck_max_30_ver.csv')
ck_std_df = pd.read_csv('ck_std_30_ver.csv')
ck_mean_df = pd.read_csv('ck_mean_30_ver.csv')

In [24]:
agglo['Instance'] = agglo['name'] + '_' + agglo['version']
agglo['Label'] = agglo['n_clusters'].astype(str)

bunch['Instance'] = bunch['name'] + '_' + bunch['version']
bunch['Label'] = bunch['bunch_algo']

In [25]:
ck_sum_df['Instance'] = ck_sum_df['project_name'] + '_' + ck_sum_df['version']
ck_max_df['Instance'] = ck_max_df['project_name'] + '_' + ck_max_df['version']

#ck_sum_df.head()

temp1 = ck_sum_df.merge(ck_max_df, on='Instance', suffixes=('_sum', '_max') )

ck_std_df['Instance'] = ck_std_df['project_name'] + '_' + ck_std_df['version']
ck_mean_df['Instance'] = ck_mean_df['project_name'] + '_' + ck_mean_df['version']

#ck_std_df.head()

temp2 = ck_sum_df.merge(ck_max_df, on='Instance', suffixes=('_std', '_mean') )

temp1 = temp1.merge(temp2, on='Instance')
agglo = agglo.merge(temp1, on='Instance')

In [26]:
agglo.head()

,name,version,n_clusters,affinity,linkage,n_MoJo,mojoFM,Instance,Label,project_name_sum,...,numbersQty_mean,assignmentsQty_mean,mathOperationsQty_mean,variablesQty_mean,maxNestedBlocks_mean,anonymousClassesQty_mean,subClassesQty_mean,lambdasQty_mean,uniqueWordsQty_mean,modifiers_mean
0,apache-storm,1.1.1,50,euclidean,single,1574,23.406326,apache-storm_1.1.1,50,apache-storm,...,613,392,57,333,7,45,593,0,1178,1025
1,apache-storm,1.1.2,50,cosine,single,508,75.279805,apache-storm_1.1.2,50,apache-storm,...,613,392,57,333,7,45,593,0,1187,1025
2,apache-storm,1.1.3,50,cosine,single,511,75.133820,apache-storm_1.1.3,50,apache-storm,...,613,392,53,333,7,45,593,0,1191,1025
3,apache-storm,1.1.4,50,cosine,single,505,75.425791,apache-storm_1.1.4,50,apache-storm,...,613,392,53,333,7,45,593,0,1191,1025
4,apache-storm,1.2.0,50,cosine,single,468,77.226277,apache-storm_1.2.0,50,apache-storm,...,613,392,57,333,7,45,593,0,1209,1025


In [27]:
columns_to_avoid = ['name', 'version', 'n_clusters', 'affinity', 'linkage', 'n_MoJo', 'mojoFM']
total_cancer = []
for element in agglo.columns:
    #print(element)
    for cancer in columns_to_avoid:
        print(cancer)
        if cancer in element:
            total_cancer.append(element)

name
version
n_clusters
affinity
linkage
n_MoJo
mojoFM
name
version
n_clusters
affinity
linkage
n_MoJo
mojoFM
name
version
n_clusters
affinity
linkage
n_MoJo
mojoFM
name
version
n_clusters
affinity
linkage
n_MoJo
mojoFM
name
version
n_clusters
affinity
linkage
n_MoJo
mojoFM
name
version
n_clusters
affinity
linkage
n_MoJo
mojoFM
name
version
n_clusters
affinity
linkage
n_MoJo
mojoFM
name
version
n_clusters
affinity
linkage
n_MoJo
mojoFM
name
version
n_clusters
affinity
linkage
n_MoJo
mojoFM
name
version
n_clusters
affinity
linkage
n_MoJo
mojoFM
name
version
n_clusters
affinity
linkage
n_MoJo
mojoFM
name
version
n_clusters
affinity
linkage
n_MoJo
mojoFM
name
version
n_clusters
affinity
linkage
n_MoJo
mojoFM
name
version
n_clusters
affinity
linkage
n_MoJo
mojoFM
name
version
n_clusters
affinity
linkage
n_MoJo
mojoFM
name
version
n_clusters
affinity
linkage
n_MoJo
mojoFM
name
version
n_clusters
affinity
linkage
n_MoJo
mojoFM
name
version
n_clusters
affinity
linkage
n_MoJo
mojoFM
name
versi

In [28]:
agglo = agglo.drop(columns=total_cancer)

In [29]:
agglo.head()

,Instance,Label,cbo_sum,wmc_sum,dit_sum,rfc_sum,lcom_sum,totalMethods_sum,staticMethods_sum,publicMethods_sum,...,numbersQty_mean,assignmentsQty_mean,mathOperationsQty_mean,variablesQty_mean,maxNestedBlocks_mean,anonymousClassesQty_mean,subClassesQty_mean,lambdasQty_mean,uniqueWordsQty_mean,modifiers_mean
0,apache-storm_1.1.1,50,13675,27066,3097,23667,117111,13332,1528,10868,...,613,392,57,333,7,45,593,0,1178,1025
1,apache-storm_1.1.2,50,13930,27294,3120,24140,117459,13488,1530,10990,...,613,392,57,333,7,45,593,0,1187,1025
2,apache-storm_1.1.3,50,13974,27385,3128,24221,118054,13522,1549,11006,...,613,392,53,333,7,45,593,0,1191,1025
3,apache-storm_1.1.4,50,13975,27391,3128,24209,118196,13524,1551,11010,...,613,392,53,333,7,45,593,0,1191,1025
4,apache-storm_1.2.0,50,14345,28032,3194,24958,119489,13964,1589,11303,...,613,392,57,333,7,45,593,0,1209,1025


In [30]:
agglo_label = agglo.pop('Label')
agglo['Label'] = agglo_label

In [32]:
agglo['Label'] = agglo['Label'].astype(str) + '_'

In [33]:
agglo.head()

,Instance,cbo_sum,wmc_sum,dit_sum,rfc_sum,lcom_sum,totalMethods_sum,staticMethods_sum,publicMethods_sum,privateMethods_sum,...,assignmentsQty_mean,mathOperationsQty_mean,variablesQty_mean,maxNestedBlocks_mean,anonymousClassesQty_mean,subClassesQty_mean,lambdasQty_mean,uniqueWordsQty_mean,modifiers_mean,Label
0,apache-storm_1.1.1,13675,27066,3097,23667,117111,13332,1528,10868,1235,...,392,57,333,7,45,593,0,1178,1025,50_
1,apache-storm_1.1.2,13930,27294,3120,24140,117459,13488,1530,10990,1248,...,392,57,333,7,45,593,0,1187,1025,50_
2,apache-storm_1.1.3,13974,27385,3128,24221,118054,13522,1549,11006,1263,...,392,53,333,7,45,593,0,1191,1025,50_
3,apache-storm_1.1.4,13975,27391,3128,24209,118196,13524,1551,11010,1262,...,392,53,333,7,45,593,0,1191,1025,50_
4,apache-storm_1.2.0,14345,28032,3194,24958,119489,13964,1589,11303,1340,...,392,57,333,7,45,593,0,1209,1025,50_


In [34]:
agglo.to_csv('agglo_remodularization_input_30_nclusters.csv', index=False)